## Shannon analysis

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Using default parameters for a local PostgreSQL instance
host = 'localhost'
port = '5432'
database = '2023_flight_data'
user = 'postgres'
password = 'XXXXXXXXXX'

# Create the database connection string
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

# Create Database Connection
engine = create_engine(connection_string)
conn = engine.connect()


In [5]:
# Query to fetch the data
query = "SELECT * FROM flight_data;"

# Load data into a pandas DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df.head())

   flight_data_id  month       carrier_name airport  \
0               1     12  Endeavor Air Inc.     ABE   
1               2     12  Endeavor Air Inc.     AEX   
2               3     12  Endeavor Air Inc.     AGS   
3               4     12  Endeavor Air Inc.     ALB   
4               5     12  Endeavor Air Inc.     ATL   

                             city  total_arrivals  total_delays_ct  \
0  Allentown/Bethlehem/Easton, PA            72.0              5.0   
1                  Alexandria, LA            62.0              7.0   
2                     Augusta, GA            95.0             10.0   
3                      Albany, NY            23.0              2.0   
4                     Atlanta, GA          2111.0            256.0   

   carrier_ct  weather_ct  nat_air_sys_ct  security_ct  late_aircraft_ct  \
0        2.46        1.00            0.73          0.0              0.81   
1        4.25        0.00            1.00          0.0              1.75   
2        5.94       

In [7]:
query_unique_carriers = """
SELECT COUNT(DISTINCT carrier_name) AS total_unique_carriers
FROM flight_data;
"""

df_unique_carriers = pd.read_sql(query_unique_carriers, engine)
print(df_unique_carriers)


   total_unique_carriers
0                     21


In [11]:
query_unique_airports = """
SELECT COUNT(DISTINCT airport) AS total_unique_airports
FROM flight_data;
"""

df_unique_airports = pd.read_sql(query_unique_airports, engine)
print(df_unique_airports)


   total_unique_airports
0                    359


In [9]:
# Total Arrivals by Carrier Name
query_total_arrivals = """
SELECT carrier_name, SUM(total_arrivals) AS total_arrivals
FROM flight_data
GROUP BY carrier_name
ORDER BY total_arrivals DESC;
"""
df_total_arrivals = pd.read_sql(query_total_arrivals, engine)
df_total_arrivals.to_csv('total_arrivals_by_carrier.csv', index=False)
print("Total Arrivals by Carrier Name:")
print(df_total_arrivals)
print()

Total Arrivals by Carrier Name:
                               carrier_name  total_arrivals
0                        Southwest Airlines       1438465.0
1                   Delta Air Lines Network        984986.0
2                 American Airlines Network        940531.0
3                  United Air Lines Network        732212.0
4                     SkyWest Airlines Inc.        675285.0
5                          Republic Airline        295651.0
6                           JetBlue Airways        274852.0
7                           Spirit Airlines        263871.0
8                   Alaska Airlines Network        245344.0
9                                 Envoy Air        227505.0
10                        Endeavor Air Inc.        201517.0
11                        PSA Airlines Inc.        194205.0
12                        Frontier Airlines        177542.0
13                            Allegiant Air        115539.0
14                        Piedmont Airlines         99047.0
15      

In [11]:
delay_types_ct = ['total_delays_ct', 'carrier_ct', 'weather_ct', 'nat_air_sys_ct', 'security_ct', 'late_aircraft_ct']
for delay_type in delay_types_ct:
    query = f"""
    SELECT carrier_name, SUM({delay_type}) AS total_{delay_type}
    FROM flight_data
    GROUP BY carrier_name
    ORDER BY total_{delay_type} DESC
    LIMIT 5;
    """
    df = pd.read_sql(query, engine)
    df.to_csv(f'top_5_airlines_by_{delay_type}_ct.csv', index=False)
    print(f"Top 5 Airlines by {delay_type}")
    print(df)
    print()


Top 5 Airlines by total_delays_ct
                carrier_name  total_total_delays_ct
0         Southwest Airlines               309963.0
1  American Airlines Network               213850.0
2    Delta Air Lines Network               159257.0
3   United Air Lines Network               148386.0
4      SkyWest Airlines Inc.               108189.0

Top 5 Airlines by carrier_ct
                carrier_name  total_carrier_ct
0         Southwest Airlines          92307.23
1    Delta Air Lines Network          65729.11
2  American Airlines Network          65391.12
3      SkyWest Airlines Inc.          61548.90
4   United Air Lines Network          50808.62

Top 5 Airlines by weather_ct
                carrier_name  total_weather_ct
0      SkyWest Airlines Inc.          11733.82
1  American Airlines Network           7415.91
2    Delta Air Lines Network           4792.08
3   United Air Lines Network           3976.50
4         Southwest Airlines           3924.71

Top 5 Airlines by nat_air_sys

In [13]:
delay_types_hr = ['total_delays_min', 'carrier_delay_min', 'weather_delay_min', 'nat_air_sys_delay_min', 'security_delay_min', 'late_aircraft_delay_min']
for delay_type in delay_types_hr:
    query = f"""
    SELECT carrier_name, ROUND(CAST(SUM({delay_type}) AS numeric) /60, 2) AS total_{delay_type}
    FROM flight_data
    GROUP BY carrier_name
    ORDER BY total_{delay_type} DESC
    LIMIT 5;
    """
    df = pd.read_sql(query, engine)
    df.to_csv(f'top_5_airlines_by_{delay_type}_hr.csv', index=False)
    print(f"Top 5 Airlines by {delay_type}")
    print(df)
    print()

Top 5 Airlines by total_delays_min
                carrier_name  total_total_delays_min
0  American Airlines Network               304923.45
1         Southwest Airlines               256699.52
2    Delta Air Lines Network               189609.53
3   United Air Lines Network               175673.60
4      SkyWest Airlines Inc.               140363.40

Top 5 Airlines by carrier_delay_min
                carrier_name  total_carrier_delay_min
0  American Airlines Network                107798.47
1    Delta Air Lines Network                 94092.05
2      SkyWest Airlines Inc.                 74959.88
3         Southwest Airlines                 71803.30
4   United Air Lines Network                 60657.40

Top 5 Airlines by weather_delay_min
                carrier_name  total_weather_delay_min
0      SkyWest Airlines Inc.                 21501.95
1  American Airlines Network                 12570.30
2    Delta Air Lines Network                  9278.15
3   United Air Lines Network     

In [15]:
delay_types_ct = ['total_delays_ct', 'carrier_ct', 'weather_ct', 'nat_air_sys_ct', 'security_ct', 'late_aircraft_ct']
for delay_type in delay_types_ct:
    query = f"""
    SELECT airport, SUM({delay_type}) AS total_{delay_type}
    FROM flight_data
    GROUP BY airport
    ORDER BY total_{delay_type} DESC
    LIMIT 5;
    """
    df = pd.read_sql(query, engine)
    df.to_csv(f'top_5_airports_by_{delay_type}_ct.csv', index=False)
    print(f"Top 5 Airports by {delay_type}")
    print(df)
    print()

Top 5 Airports by total_delays_ct
  airport  total_total_delays_ct
0     DEN                62805.0
1     DFW                60305.0
2     ORD                56837.0
3     ATL                55727.0
4     LAS                48933.0

Top 5 Airports by carrier_ct
  airport  total_carrier_ct
0     ORD          19094.21
1     DEN          19034.14
2     ATL          18557.79
3     DFW          17575.99
4     LAX          14827.17

Top 5 Airports by weather_ct
  airport  total_weather_ct
0     DFW           3277.44
1     DEN           2297.23
2     ATL           2196.69
3     ORD           2042.29
4     CLT           1661.55

Top 5 Airports by nat_air_sys_ct
  airport  total_nat_air_sys_ct
0     DEN              19064.38
1     LAS              16866.96
2     ORD              16407.97
3     DFW              16094.93
4     EWR              13851.60

Top 5 Airports by security_ct
  airport  total_security_ct
0     DFW             232.50
1     ANC             188.58
2     LAX             168.87

In [17]:
delay_types_hr = ['total_delays_min', 'carrier_delay_min', 'weather_delay_min', 'nat_air_sys_delay_min', 'security_delay_min', 'late_aircraft_delay_min']
for delay_type in delay_types_hr:
    query = f"""
    SELECT airport, ROUND(CAST(SUM({delay_type}) AS numeric) /60, 2) AS total_{delay_type}
    FROM flight_data
    GROUP BY airport
    ORDER BY total_{delay_type} DESC
    LIMIT 5;
    """
    df = pd.read_sql(query, engine)
    df.to_csv(f'top_5_airports_by_{delay_type}_hr.csv', index=False)
    print(f"Top 5 Airports by {delay_type}")
    print(df)
    print()

Top 5 Airports by total_delays_min
  airport  total_total_delays_min
0     DFW                89335.90
1     ORD                76050.90
2     DEN                74204.33
3     ATL                70323.02
4     CLT                59510.08

Top 5 Airports by carrier_delay_min
  airport  total_carrier_delay_min
0     DFW                 32983.10
1     ATL                 29477.38
2     ORD                 28216.98
3     DEN                 23809.18
4     CLT                 20535.80

Top 5 Airports by weather_delay_min
  airport  total_weather_delay_min
0     DFW                  5891.98
1     DEN                  4350.77
2     ORD                  4230.62
3     ATL                  4030.40
4     CLT                  3034.68

Top 5 Airports by nat_air_sys_delay_min
  airport  total_nat_air_sys_delay_min
0     DEN                     17485.87
1     LAS                     15026.65
2     ORD                     13869.70
3     EWR                     13674.35
4     MCO                     1

In [19]:
# Close the connection
conn.close()